# Tensorboard Basics

Graph and Loss visualization using Tensorboard.

In [3]:
import tensorflow as tf

# Import MINST data
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [4]:
# Parameters
learning_rate = 0.01
training_epochs = 25
batch_size = 100
display_epoch = 1
logs_path = '/tmp/tensorflow_logs/example/'

# tf Graph Input
# mnist data image of shape 28*28=784
x = tf.placeholder(tf.float32, [None, 784], name='InputData')
# 0-9 digits recognition => 10 classes
y = tf.placeholder(tf.float32, [None, 10], name='LabelData')

# Set model weights
W = tf.Variable(tf.zeros([784, 10]), name='Weights')
b = tf.Variable(tf.zeros([10]), name='Bias')

In [5]:
# Construct model and encapsulating all ops into scopes, making
# Tensorboard's Graph visualization more convenient
with tf.name_scope('Model'):
    # Model
    pred = tf.nn.softmax(tf.matmul(x, W) + b) # Softmax
with tf.name_scope('Loss'):
    # Minimize error using cross entropy
    cost = tf.reduce_mean(-tf.reduce_sum(y * tf.log(pred), reduction_indices=1))
with tf.name_scope('SGD'):
    # Gradient Descent
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)
with tf.name_scope('Accuracy'):
    # Accuracy
    acc = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
    acc = tf.reduce_mean(tf.cast(acc, tf.float32))

# Initializing the variables
init = tf.global_variables_initializer()

# Create a summary to monitor cost tensor
tf.summary.scalar("loss", cost)
# Create a summary to monitor accuracy tensor
tf.summary.scalar("accuracy", acc)
# Merge all summaries into a single op
merged_summary_op = tf.summary.merge_all()

In [9]:
# Start Training
with tf.Session() as sess:
    sess.run(init)

    # op to write logs to Tensorboard
    summary_writer = tf.summary.FileWriter(logs_path, graph=tf.get_default_graph())

    # Training cycle
    for epoch in range(training_epochs):
        avg_cost = 0.
        total_batch = int(mnist.train.num_examples / batch_size)
        # Loop over all batches
        for i in range(total_batch):
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)
            # Run optimization op (backprop), cost op (to get loss value)
            # and summary nodes
            _, c, summary = sess.run([optimizer, cost, merged_summary_op],
                                     feed_dict={x: batch_xs, y: batch_ys})
            # Write logs at every iteration
            summary_writer.add_summary(summary, epoch * total_batch + i)
            # Compute average loss
            avg_cost += c / total_batch
        # Display logs per epoch step
        if (epoch+1) % display_epoch == 0:
            print(f"Epoch: {epoch+1:4}, cost={avg_cost:.9f}")

    print("Optimization Finished!")

    # Test model
    # Calculate accuracy
    print(f"Accuracy: {acc.eval({x: mnist.test.images, y: mnist.test.labels})}")

    print("Run the command line:\n" \
          "--> tensorboard --logdir=/tmp/tensorflow_logs " \
          "\nThen open http://0.0.0.0:6006/ into your web browser")

Epoch:    1, cost=1.182840493
Epoch:    2, cost=0.668031614
Epoch:    3, cost=0.549015398
Epoch:    4, cost=0.499624828
Epoch:    5, cost=0.463698757
Epoch:    6, cost=0.443981911
Epoch:    7, cost=0.424893631
Epoch:    8, cost=0.412020342
Epoch:    9, cost=0.402691247
Epoch:   10, cost=0.389852398
Epoch:   11, cost=0.389641608
Epoch:   12, cost=0.373782504
Epoch:   13, cost=0.373548098
Epoch:   14, cost=0.369599616
Epoch:   15, cost=0.361204420
Epoch:   16, cost=0.357332581
Epoch:   17, cost=0.356536867
Epoch:   18, cost=0.353244983
Epoch:   19, cost=0.345658542
Epoch:   20, cost=0.345977435
Epoch:   21, cost=0.345181416
Epoch:   22, cost=0.337512340
Epoch:   23, cost=0.339612240
Epoch:   24, cost=0.335536570
Epoch:   25, cost=0.333478308
Optimization Finished!
Accuracy: 0.9128999710083008
Run the command line:
--> tensorboard --logdir=/tmp/tensorflow_logs 
Then open http://0.0.0.0:6006/ into your web browser
